In [1]:
%pylab inline
import tensorflow as tf
from src.setup.config import db
from itertools import groupby
import pandas as pd
from collections import OrderedDict
from datetime import date
import pyodbc

Populating the interactive namespace from numpy and matplotlib


C:\Users\Vivi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'src'

In [ ]:
#from tensorflow.contrib.tensorboard.plugins import projector
tf.reset_default_graph()
%matplotlib notebook
learning_rate = 0.05
training_epochs = 10000
batch_size = 50
display_step = 1
# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 9999 # data input 
n_classes = dictsize # total classes (0-9999 drgs)
step = 0

# tf Graph input

#X = tf.placeholder("float", [None,n_input])
X = tf.placeholder("int32", [None,numICDs])
Y = tf.placeholder("int32",[None,1])


keep_probability = tf.placeholder("float")
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
# Create model

def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(inputVec, weights['h1']), biases['b1'])),keep_probability)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])),keep_probability)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
out = multilayer_perceptron(X)
#Define accuracy (average number of correctly predicted treatments (only trues count))
pred = tf.argmax(tf.nn.softmax(out),axis=1,output_type=tf.int32)
Comparison=tf.equal(tf.reshape(pred,(batch_size,1)),Y)
RightPred=tf.reduce_sum(tf.reduce_sum(tf.cast(Comparison, tf.float32),1))/batch_size

diff = tf.reduce_mean(tf.losses.absolute_difference(tf.reshape(pred,(batch_size,1)),Y))
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=out, labels=targetVec))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


tf.summary.scalar("cross_entropy", loss_op)
tf.summary.scalar("accuracy",RightPred)
tf.summary.scalar("absDiff",diff)

merged_summaries = tf.summary.merge_all()
# Initializing the variables
#init = tf.global_variables_initializer()

train_writer = tf.summary.FileWriter("./DRGsummaries/trainICDtoDict", tf.get_default_graph())
validation_writer = tf.summary.FileWriter("./DRGsummaries/valICDtoDict")


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    kp=0.8
    for n in range(training_epochs):   
        icd, ops = getBatch(batch_size,"train",'dictionary')
        _,loss,_summaries = sess.run([train_op,loss_op,merged_summaries], feed_dict={X: icd,Y: np.reshape(ops,(batch_size,1)),keep_probability:kp})             

        train_writer.add_summary(_summaries, step)
        if (step % 50 == 0): 
            kp=1.0
            icd, ops = getBatch(batch_size,"val",'dictionary')
            _,acc,loss_val,_summaries,absdiff = sess.run([train_op,RightPred,loss_op,merged_summaries,diff], feed_dict={X: icd,
                                                            Y: np.reshape(ops,(batch_size,1)),keep_probability:kp})
            print("Accuracy: ",acc)
            print("Cross Entropy: ",loss_val)
            print("Diff: ",absdiff)

            validation_writer.add_summary(_summaries, step)
        #if (step%1000==0):
         #   saver.save(sess,"./log/predDRGfromICDDict.ckpt")
                 
        step = step + 1